In [1]:
# 구글 드라이브의 파일 연동
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# 코랩에서 접근하고 싶은 파일들을 먼저 압축하고
# 구글 드라이브에 업로드 한 후
# 코랩과 구글 드라이브를 연동시키고
# 현재 화면의 좌측 부분의 파일 아이콘을 선택하여
# 해당 파일을 찾아 우클릭 하고 경로 복사 후 압축 해제
# !unzip "/content/drive/MyDrive/open.zip"

In [4]:
import os
from datetime import datetime
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import Xception
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense

In [5]:
# ========= 코드 그대로 모델 생성하고 학습시켜보기 =========
# https://techblog-history-younghunjo1.tistory.com/261

TRAIN_FOLDER_PATH = '/content/train/'
TEST_FOLDER_PATH = '/content/test/'
# TRAIN_FOLDER_PATH = '../../open/train/'
# TEST_FOLDER_PATH = '../../open/test/'

EPOCHS = 5
RESIZED_WIDTH = 224
RESIZED_HEIGHT = 224

train_img_generator = ImageDataGenerator(rotation_range=30 ,width_shift_range=0.3, shear_range=0.3, horizontal_flip = True, vertical_flip = True, rescale=1/255.)
train_img_flow = train_img_generator.flow_from_directory(
    TRAIN_FOLDER_PATH, target_size=(RESIZED_WIDTH, RESIZED_HEIGHT))

# 추론 데이터 X
# test_img_generator = ImageDataGenerator(rescale=1/255.)
# test_img_flow = test_img_generator.flow_from_directory(
#     TRAIN_FOLDER_PATH, target_size=(RESIZED_WIDTH, RESIZED_HEIGHT))

# 109
print(len(train_img_flow))

# for img in train_img_flow:
#     print('img :', img)

def create_model_fn():

    input_tensor = Input(shape=(RESIZED_WIDTH, RESIZED_HEIGHT, 3))

    pretrained_model = Xception(
        input_tensor=input_tensor, include_top=False, weights='imagenet')
    pretrained_output = pretrained_model.output

    x = GlobalAveragePooling2D()(pretrained_output)
    x = Dense(units=128, activation='relu')(x)
    output = Dense(units=19, activation='softmax')(x)

    model = Model(inputs=input_tensor, outputs=output)
    model.summary()
    return model

model = create_model_fn()
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy', metrics=['acc'])

train_history = model.fit(train_img_flow, epochs=EPOCHS)
loss = train_history.history['loss']
print('loss :', loss)

# 연월일_시간
now = datetime.today().strftime('%Y%m%d_%H%M%S')
model.save(f'model_{now}.h5')

# ========================================================


Found 3457 images belonging to 19 classes.
109
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 111, 111, 32  128        ['block1_conv1[0][0]']           
 ation)                         )              